

```
# This is formatted as code
```

Exercise 2: Train a VAE to generate images similar to the images of the given fashion-mnist dataset. 

The dataset is given in two csv files (train and test). ALL data should be used to train the VAE. The csv files have headers, their first columns contain labels, and the remaining columns contain image data.

Your should: write dataloaders, train an appropriate VAE, show a few samples and their corresponding decoded images, as well as a few of the generated samples.


In [ ]:
!wget -O data.zip https://www.dropbox.com/s/tzs174hrjs1rbzs/FashionMNIST.zip?dl=0
!unzip data.zip

--2023-01-16 16:56:19--  https://www.dropbox.com/s/tzs174hrjs1rbzs/FashionMNIST.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/tzs174hrjs1rbzs/FashionMNIST.zip [following]
--2023-01-16 16:56:19--  https://www.dropbox.com/s/raw/tzs174hrjs1rbzs/FashionMNIST.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc21e8eb1102cb91293ef433f7f8.dl-eu.dropboxusercontent.com/cd/0/inline/B0q3tqiCpLKzsfD_Q2U8HOPGd4wHCWO7mBVIN6NohPAurmaV0Kox23X_igoiZUItfZ4WbKWTmaqIpmdYoI3QR1Nw4RU_-67PeaC8xk5b2ce1wlk0oNMttQjhJZsxliGpczkH4McQSvA99v61KN3_yQN0MiinPU1z8gbZctxNe_1SuQ/file# [following]
--2023-01-16 16:56:19--  https://uc21e8eb1102cb91293ef433f7f8.dl-eu.dropboxusercontent.com/cd/0/inline/B0q3tqiCpLKzsfD_Q2U8HOPGd4wHCWO7mBVIN6NohPAurmaV0Kox23X_i

In [ ]:
import pandas as pd
import numpy 
df1=pd.read_csv("/content/fashion-mnist_test.csv" , header=None)
df2=pd.read_csv("/content/fashion-mnist_train.csv" , header=None)
df1=df1.to_numpy()
df2=df2.to_numpy()
df1=numpy.delete(df1,0,axis=0)
df1=numpy.delete(df1,0,axis=1)
df2=numpy.delete(df2,0,axis=0)
df2=numpy.delete(df2,0,axis=1)
DATA = numpy.concatenate((df1,df2))
# df3=pd.concat([df1,df2],axis=1)
# df3.to_csv("../content/sample_data/all.csv",index=False) #save to file


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [ ]:
DATA

array([['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [ ]:
import torchvision.transforms as transform
from torch.utils.data import Dataset
import torch


class VAN(Dataset):
  def __init__(self, File):
    super().__init__()
    
    self.X = File.astype(numpy.float32).reshape(-1,28,28)/255.0  # creaing the image from vectors to image . -1 i don't know the number of the sample
    # self.Y=df_y.iloc[:,0].to_numpy()

  def __len__(self):
    return len(self.X)

  def __getitem__ (self,idx):
    image= self.X[idx]
    image = torch.tensor(numpy.expand_dims(image,axis=0))


    # lable = torch.tensor(self.Y[idx])
    return image
    # ,lable


In [ ]:
trainset = VAN(DATA)
batch_sz = 64
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_sz, shuffle=True)
#Iterater 
dataiter = iter(train_loader)
images = next(dataiter)

#images.shape = Batch size X GreyScale (Single value per pixel) X #Pixels X #Pixels
#labels.shape = ????
print(images.shape)
type(images[0])

torch.Size([64, 1, 28, 28])


torch.Tensor

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class VarAutoEncoder(nn.Module):
  def __init__(self, nz):
    super().__init__()
    self.nz = nz
    self.econv1 = nn.Conv2d(1, 16, 3, 2, 1)
    self.ebn1 = nn.BatchNorm2d(16)
    self.econv2 = nn.Conv2d(16, 32, 3, 2, 1)
    self.ebn2 = nn.BatchNorm2d(32)
    self.econv3 = nn.Conv2d(32,64, 3, 2, 1)
    self.ebn3 = nn.BatchNorm2d(64)
    self.elinear1 = nn.Linear(64*4*4, 128)
    self.ebn4 = nn.BatchNorm1d(128)
    
    self.dlinear1 = nn.Linear(nz,128)
    self.dbn1 = nn.BatchNorm1d(128)
    self.dlinear2 = nn.Linear(128,64*4*4)
    self.dbn2 = nn.BatchNorm2d(64)
    self.dconv1 = nn.ConvTranspose2d(64, 32, 3, 2, 1)
    self.dbn3 = nn.BatchNorm2d(32)
    self.dconv2 = nn.ConvTranspose2d(32, 16, 3, 2, 1, output_padding =1)
    self.dbn4 = nn.BatchNorm2d(16)
    self.dconv3 = nn.ConvTranspose2d(16, 1, 3, 2, 1, output_padding =1)
    
    self.ulinear = nn.Linear(128,nz)
    self.slinear = nn.Linear(128,nz)


  def encoder (self,x):
    x=x.view(-1,1,28,28)
    print("i am here")
    print(x.shape)
    x = F.leaky_relu(self.econv1(x))
    print("i am here2")

    x = self.ebn1(x)
    x = F.leaky_relu(self.econv2(x))
    print("i am here3")

    x = self.ebn2(x)
    x = F.leaky_relu(self.econv3(x))
    x = self.ebn3(x)
    x = x.view(-1,64*4*4)
    print("i am here5")

    x = F.leaky_relu(self.elinear1(x))
    print("here is the shape of it ")
    print(x.shape)

    x = self.ebn4(x)


    return x
  
  def decoder(self, x):
    x = F.leaky_relu(self.dlinear1(x))
    x = self.dbn1(x)
    x = F.leaky_relu(self.dlinear2(x))
    x = x.view(-1, 64,4,4)
    x = self.dbn2(x)
    x = F.leaky_relu(self.dconv1(x))
    x = self.dbn3(x)
    x = F.leaky_relu(self.dconv2(x))
    x = self.dbn4(x)
    x = torch.sigmoid(self.dconv3(x))
    return x

  def get_params(self,x):
    u = self.ulinear(x)
    logvar = self.slinear(x)
    return u, logvar

  def reparametarization(self, u, logvar):
    std = torch.exp(logvar/2)
    z = u + std*torch.randn_like(u)
    return z

  def forward(self,x):
    ex = self.encoder(x)
    print("i am here99")

    u, logvar = self.get_params(ex)
    print("i am here909")

    z = self.reparametarization(u, logvar)

    x = self.decoder(z)
    return ex,z, x, u, logvar 

In [ ]:
from torchsummary import summary
device = "cuda" if torch.cuda.is_available() else "cpu"

net = VarAutoEncoder(32)
net = VarAutoEncoder(32).to(device)
summary(net,(1,28,28))
# for batch in train_loader:
#     X = batch[0][0].to(device)
#     print(X)


i am here
torch.Size([2, 1, 28, 28])
i am here2
i am here3
i am here5
here is the shape of it 
torch.Size([2, 128])
i am here99
i am here909
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 14, 14]             160
       BatchNorm2d-2           [-1, 16, 14, 14]              32
            Conv2d-3             [-1, 32, 7, 7]           4,640
       BatchNorm2d-4             [-1, 32, 7, 7]              64
            Conv2d-5             [-1, 64, 4, 4]          18,496
       BatchNorm2d-6             [-1, 64, 4, 4]             128
            Linear-7                  [-1, 128]         131,200
       BatchNorm1d-8                  [-1, 128]             256
            Linear-9                   [-1, 32]           4,128
           Linear-10                   [-1, 32]           4,128
           Linear-11                  [-1, 128]           4,224
      BatchNorm1d-12      

In [ ]:
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"
net = VarAutoEncoder(32).to(device) # you need to send the model to the devoic

num_epochs = 30
lr = 0.0001
mm = 0.5
optimizer = optim.Adam(net.parameters(), lr = lr)
ls = []
#(reconstruction loss )+(regulaztion)
def var_loss(Xhat, X, u, logvar): 
  KLD_WEIGHT = 0.0012
  term1 = F.mse_loss(Xhat, X) #reconstruction loss mean sqaured error
  term2 = torch.mean(torch.sum(0.5*(1+logvar - torch.exp(logvar) - u**2), axis = 1), axis =0)
  return (1- KLD_WEIGHT)* term1 - KLD_WEIGHT*term2 #we weight them what do i care more about to give more wight
torch.Size([64, 1, 28, 28])
torch.Size([28, 28])

for epoch in range(num_epochs):
  net.train()
  total_loss = 0
  for batch in train_loader:
    
    X = batch[0].to(device)
    print("hi")
    print(X.shape)
    ex, z,Xhat, u, logvar = net(X)
    loss = var_loss(Xhat, X,u, logvar )
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(epoch, total_loss)

  ls.append(total_loss)

plt.plot(ls)


hi
torch.Size([1, 28, 28])
i am here
torch.Size([1, 1, 28, 28])
i am here2
i am here3
i am here5
here is the shape of it 
torch.Size([1, 128])


ValueError: ignored

In [ ]:
net = VarAutoEncoder(3).to(device) # you need to send the model to the devoic
net.eval() #no using of the mean of the sample use the one compute intraing 
#take a random z and then decode that z 
z = torch.randn(64,3).to(device) #64 vacotrs with size of 32 in a one same as the  laten sizze of the image 
Xhat = net.decoder(z) #64*1*28*28


In [ ]:
img = Xhat[0, 0]
print(Xhat.shape, img.shape, img.dtype)
plt.imshow(img.detach().cpu().numpy(), cmap='gray')